In [ ]:
import requests
import json
import pandas as pd
import time

In [ ]:
#Retrieving a list of account ID's based on first three letters of player's nickname

In [ ]:
three_letters = ['nic', 'das', 'jee', 'kvi', 'ddt', 'tec', 'ein', 'pek', 'zed', 'bot', 'maw', 'egl', 
                       'kej', 'cab', 'vyb', 'abc', 'fik', 'hit', 'ill', 'lak', 'siu', 'ulj']

In [ ]:
base_url = "https://api.worldoftanks.eu/wot/account/list/?application_id="
application_id = '67369ded9212f123394343e566e'

player_list = []

for i in three_letters:
    response = requests.get(base_url + application_id + '&type=startswith&search=' + i)
    iload = response.json()
    player_list.extend(iload['data'])    

In [ ]:
account_ids_str = [str(d['account_id']) for d in player_list]

In [ ]:
#Splitting account id's to 100 per chunk and joining them to use in request URL's
#(API allows only 100 id's per one request)

In [ ]:
two_d_list = [account_ids_str[x:x + 100] for x in range(0, len(account_ids_str), 100)]

In [ ]:
string_chunk = ['%2C'.join(i) for i in two_d_list]

In [ ]:
string_chunk

In [ ]:
#Now getting the player data and storing in JSON

In [ ]:
player_data = []

In [ ]:
url = "https://api.worldoftanks.eu/wot/account/info/?application_id="
application_id = '67369ded9212f123394343e566ee'
params = '&fields=account_id%2C+created_at%2C+global_rating%2C+statistics.all.battles%2C+statistics.all.max_damage%2C+statistics.all.max_damage_tank_id%2C+statistics.all.max_frags%2C+statistics.all.max_frags_tank_id%2C+statistics.all.max_xp%2C+statistics.all.max_xp_tank_id%2C+statistics.all.shots%2C+statistics.all.wins&account_id='
for i in string_chunk:
    response1 = requests.get(url + application_id + params + i)
    item_load = response1.json()
    player_data.append(item_load)    
    
    time.sleep(1.5)

In [ ]:
# Flattening the JSON

In [ ]:
pre_flattened = {}
for i in range(len(player_data)):
    pre_flattened.update(player_data[i])

In [ ]:
flattened = []
for acc_id in account_ids_str:
    flattened.append(pre_flattened[acc_id])

In [ ]:
#Exploring and cleaning data

In [ ]:
data = pd.json_normalize(flattened)
data.to_csv('Player-data.csv', sep = ',', index = False)

In [ ]:
df = pd.read_csv('Player-data.csv' , index_col = 'account_id')
df_copy = df.copy()

In [ ]:
df_copy.info()

In [ ]:
df_copy.describe()

In [ ]:
pd.isna(df_copy).sum()

In [ ]:
df_copy.columns = df_copy.columns.str.removeprefix('statistics.all.').to_list()

In [ ]:
df_copy['created_at'] = pd.to_datetime(df_copy['created_at'], unit = 's').dt.date

In [ ]:
df_copy.fillna(0, inplace=True)

In [ ]:
df_copy.to_csv('Player-Data-clean.csv', sep= ',')

In [ ]:
#Retrieving Player Vehicle data

In [ ]:
vehicle_data = {}

In [ ]:
url = "https://api.worldoftanks.eu/wot/account/tanks/?application_id="
application_id = '67369ded9212f123394343e566ee'

for i in string_chunk:
    response1 = requests.get(url + application_id + '&account_id=' + i)
    item_load = response1.json()
    vehicle_data.update(item_load['data'])    
    
    time.sleep(1.5)

In [ ]:
with open("Vehicle-data.json", "w") as outfile:
    json.dump(vehicle_data, outfile, indent = 2)

In [ ]:
#Flattening JSON

In [ ]:
with open("Vehicle-data.json", "r") as file:
    data = json.load(file)

In [ ]:
list_of_keys = [i for i in data]

In [ ]:
df=[]
keys = []
for i in list_of_keys:
    for j in range(len(data[i])):
        df.append(data[i][j])
        keys.append(str(i))

In [ ]:
clean_df = pd.json_normalize(df)
clean_df['account_keys'] = keys

In [ ]:
clean_df.set_index('account_keys', inplace = True)

In [ ]:
clean_df.to_csv('Playe-Vehicle-data-clean.csv', sep = ',')